This dataset is from Kaggle (Links to an external site.) and contains employee metrics such as satisfaction level, number of projects completed, time spent at the company, and number of promotions. In a hypothetical scenario, we could use this data to determine whether or not an employee is eligible for a bonus. This dataset has four variables, each using different numerical scales and units. If we want to use this dataset for a neural network model, we need to standardize these variables, otherwise it is likely that "Satisfaction_Level" (which ranges between 0 and 1) would be undervalued compared to "Time_Spent" (which ranges from approximately 50 to 4,800 hours). 

In [1]:
#dependencies
import pandas as pd
from sklearn.preprocessing import StandardScaler

#read data
hrDF = pd.read_csv("https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-online/module_19/hr_dataset.csv")
hrDF.head()

,Satisfaction_Level,Num_Projects,Time_Spent,Num_Promotions
0,0.30,1,253,2
1,0.25,1,200,0
2,0.90,4,2880,5
3,0.65,3,1450,3
4,0.50,2,785,2


To apply our standardization, we need to create a StandardScaler instance and then fit the input data. After fitting, we can transform and standardize the dataset. Lastly, once we have our transformed data within the StandardScaler instance, we must export the transformed data into a Pandas DataFrame. 

In [2]:
#scaler instance
scaler=StandardScaler()

#fit scaler
scaler.fit(hrDF)

#xform/standardarize dataset
scaledData = scaler.transform(hrDF)

#create dataframe of scaled data
xformed_scaled_data = pd.DataFrame(scaledData, columns=hrDF.columns)
xformed_scaled_data.head()

,Satisfaction_Level,Num_Projects,Time_Spent,Num_Promotions
0,-1.303615,-1.162476,-1.049481,-0.558656
1,-1.512945,-1.162476,-1.094603,-1.804887
2,1.208335,0.860233,1.187080,1.310692
3,0.161689,0.185996,-0.030385,0.064460
4,-0.466299,-0.488240,-0.596549,-0.558656


Looking at the results of our transformation, all of the variables have now been standardized, with a mean value of 0 and a standard deviation of 1. Now the data is ready to be passed along to our neural network model.

Neural networks are designed to calculate weights for various input data and pass along an output value based on an activation function. With our basic neural networks, input data is parsed using an input layer, evaluated in a single hidden layer, then calculated in the output layer. In other words, a basic neural network is designed such that the input values are evaluated *only once* before they are used in a classification or regression equation. Although basic neural networks are relatively easy to conceptualize and understand, there are limitations to using a basic neural network, such as:

    - A basic neural network with many neurons will require more training data than other comparable statistics and machine learning models to produce an adequate model.
    - Basic neural networks struggle to interpret complex nonlinear numerical data, or data with many confounding factors that have hidden effects on more than one variable.
    - Basic neural networks are incapable of analyzing image datasets without severe data preprocessing.

To address the limitations of the basic neural network, we can implement a more robust neural network model by adding additional hidden layers. A neural network with more than one hidden layer is known as a **deep neural network**:


Deep neural networks function similarly to the basic neural network, with one major exception. The outputs of one hidden layer of neurons (that have been evaluated and transformed using an activation function) become the inputs to additional hidden layers of neurons. As a result, the next layer of neurons can evaluate higher order interactions between weighted variables and identify complex, nonlinear relationships across the entire dataset. These additional layers can observe and weight interactions between clusters of neurons across the entire dataset, which means they can identify and account for more information than any number of neurons in a single hidden layer.

Deep neural network models also are commonly referred to as **deep learning models** due to their ability to learn from example data, regardless of the complexity or data input type. Just like humans, deep learning models can identify patterns, determine severity, and adapt to changing input data from a wide variety of data sources. Compared to basic neural network models, which require a large number of neurons to identify nonlinear characteristics, deep learning models only need a few neurons across a few hidden layers to identify the same nonlinear characteristics.

For our first deep learning example, we'll look at another human resources (HR) dataset from [IBM's HR department](https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-online/module_19/HR-Employee-Attrition.csv) (Links to an external site.). This dataset contains employee career profile and attrition (departure from company) information, such as age, department, job role, work/life balance, and so forth. Using this data, we can generate a deep learning model that can help to identify whether or not a person is likely to depart from the company given his or her current employee profile.

This dataset has more than 30 variables, each with different numerical distributions and categorical variables, as well as some dichotomous variables that will need transformation.

This will be the deeplearning tabular portion.

In [3]:
#import other dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf

#import dataset

attritionDF = pd.read_csv("https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-online/module_19/HR-Employee-Attrition.csv")
attritionDF.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


Looking at the top of our DataFrame, there are multiple columns with categorical values as well as our numerical values. To make things easier, we should generate a list of categorical variable names. Instead of searching across all 35 columns and keeping track of which variables need categorical preprocessing, we'll let Python do all of the heavy lifting. As an added bonus, we can use our variable list to perform the one-hot encoding once, rather than for each individual variable. To generate our variable list, we'll use Pandas Dataframe.dtypes property. 

In [4]:
#generate categorical variable list
attrCat = attritionDF.dtypes[attritionDF.dtypes == "object"].index.tolist()
attrCat

['Attrition',
 'BusinessTravel',
 'Department',
 'EducationField',
 'Gender',
 'JobRole',
 'MaritalStatus',
 'Over18',
 'OverTime']

Now that we have our variable names separated, we can start to preprocess our data starting with the one-hot encoding of the categorical data. Looking at our `attrition_cat` variable, there are eight categorical variables that need encoding. However, before we loop through our variables and encode them using Scikit-learn's `OneHotEncoder` module, we need to make sure that none of the categorical variables have more than 10 unique values and require bucketing.

In [5]:
#chekc number of unique variables
attritionDF[attrCat].nunique()

Attrition         2
BusinessTravel    3
Department        3
EducationField    6
Gender            2
JobRole           9
MaritalStatus     3
Over18            1
OverTime          2
dtype: int64

According to the `nunique` method, none of the categorical variables have more than 10 unique values, which means we're ready to encode. Previously, we have encoded a single variable at a time, but Scikit-learn is flexible enough to perform all of the one-hot encodings at the same time. The only difference from our single variable example is we need to pass our `attrition_cat` variable list.

In [6]:
#create onehotencoder instance
enc = OneHotEncoder(sparse=False)

#fit/xform
encDF = pd.DataFrame(enc.fit_transform(attritionDF[attrCat]))

#add names to df
encDF.columns = enc.get_feature_names_out(attrCat)
encDF.head()


,Attrition_No,Attrition_Yes,BusinessTravel_Non-Travel,BusinessTravel_Travel_Frequently,BusinessTravel_Travel_Rarely,Department_Human Resources,Department_Research & Development,Department_Sales,EducationField_Human Resources,EducationField_Life Sciences,...,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,Over18_Y,OverTime_No,OverTime_Yes
0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
1,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
2,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
3,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
4,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


Now that our categorical variables have been encoded, they are ready to replace our unencoded categorical variables in our dataset. To replace these columns, we'll use a combination of Pandas' `merge` and `drop` methods.

In [7]:
#merge onehot features and drop originals
attritionDF = attritionDF.merge(encDF, left_index=True, right_index=True).drop(attrCat,1)
attritionDF.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


,Age,DailyRate,DistanceFromHome,Education,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,...,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,Over18_Y,OverTime_No,OverTime_Yes
0,41,1102,1,2,1,1,2,94,3,2,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
1,49,279,8,1,1,2,3,61,2,2,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
2,37,1373,2,2,1,4,4,92,2,1,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
3,33,1392,3,4,1,5,4,56,3,1,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
4,27,591,2,1,1,7,1,40,3,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


o build our training and testing datasets, we need to separate two values:

- input values (which are our independent variables commonly referred to as **model features** or "X" in [TensorFlow documentation](https://www.tensorflow.org/guide/))
- target output (our dependent variable commonly referred to as **target** or "y" in TensorFlow documentation)

For our purposes, we want to build a model that will predict whether or not a person is at risk for attrition; therefore, we must separate the "Attrition" columns from the rest of the input data. In fact, because the attrition data is dichotomous (one of two values), we only need to keep the "Attrition_Yes" column—we can ignore the "Attrition_No" column because it is redundant.

In [8]:
# Split our preprocessed data into our features and target arrays
y = attritionDF["Attrition_Yes"].values
X = attritionDF.drop(["Attrition_Yes", "Attrition_No"],1).values


# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=78)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


Now that our training and testing data have been allocated, we're ready to build our StandardScalerobject and standardize the numerical features, which again means that all values become get scaled to fall within one standard deviation of the mean, which becomes 0

In [9]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

Now that our data is preprocessed via one-hot encoding and standardization, we should probably perform a gut check to ensure that no data has been lost from our original DataFrame.  t last, our data is preprocessed and separated and ready for modelling. For our purposes, we will use the same framework we used for our basic neural network:

- For our input layer, we must add the number of input features equal to the number of variables in our feature DataFrame.
- In our hidden layers, our deep learning model structure will be slightly different—we'll add two hidden layers with only a few neurons in each layer. To create the second hidden layer, we'll add another Keras Dense class while defining our model. All of our hidden layers will use the relu activation function to identify nonlinear characteristics from the input values.
- In the output layer, we'll use the same parameters from our basic neural network including the sigmoid activation function. The sigmoid activation function will help us predict the probability that an employee is at risk for attrition.


In [10]:
# Define the model - deep neural net
inputFeatures = len(X_train[0])
hiddenNodes_layer1 = 8
hiddenNodes_layer2 = 5

#instantiate neural network keras model
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hiddenNodes_layer1, input_dim=inputFeatures, activation="relu")
)

# Second hidden layer, don't have to pass the input_dim argument because it is part of the first layer
nn.add(
    tf.keras.layers.Dense(units=hiddenNodes_layer2, activation="relu")
)

# Output layer
nn.add(
    tf.keras.layers.Dense(units=1, activation="sigmoid")
)


# Check the structure of the model
nn.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 448       
                                                                 
 dense_1 (Dense)             (None, 5)                 45        
                                                                 
 dense_2 (Dense)             (None, 1)                 6         
                                                                 
Total params: 499
Trainable params: 499
Non-trainable params: 0
_________________________________________________________________


Looking at our model summary, we can see that the number of weight parameters (weight coefficients) for each layer equals the number of input values times the number of neurons plus a bias term for each neuron. Our first layer has 55 input values, and multiplied by the eight neurons (plus eight bias terms for each neuron) gives us a total of 448 weight parameters—plenty of opportunities for our model to find trends in the dataset. If there are eight neurons in the first layer, and five neurons in the second layer, then there are 45 parameters; eight times five equals 40 parameters, plus five parameters for the bias terms.

Now it is time to compile our model and define the loss and accuracy metrics. Since we want to use our model as a binary classifier, we'll use the `binary_crossentropy` loss function, `adam` optimizer, and `accuracy` metrics, which are the same parameters we used for our basic neural network. 

In [11]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [12]:
#train the model
fit_model = nn.fit(X_train, y_train, epochs=100)

Epoch 1/100
35/35 [==============================] - 2s 4ms/step - loss: 85.2717 - accuracy: 0.8240
Epoch 2/100
35/35 [==============================] - 0s 3ms/step - loss: 16.1372 - accuracy: 0.7795
Epoch 3/100
35/35 [==============================] - 0s 3ms/step - loss: 4.9424 - accuracy: 0.7350
Epoch 4/100
35/35 [==============================] - 0s 5ms/step - loss: 1.5193 - accuracy: 0.6434
Epoch 5/100
35/35 [==============================] - 0s 5ms/step - loss: 0.9704 - accuracy: 0.6760
Epoch 6/100
35/35 [==============================] - 0s 7ms/step - loss: 0.8556 - accuracy: 0.6642
Epoch 7/100
35/35 [==============================] - 0s 5ms/step - loss: 0.8086 - accuracy: 0.7650
Epoch 8/100
35/35 [==============================] - 0s 3ms/step - loss: 0.7774 - accuracy: 0.8258
Epoch 9/100
35/35 [==============================] - 0s 3ms/step - loss: 0.7656 - accuracy: 0.8267
Epoch 10/100
35/35 [==============================] - 0s 3ms/step - loss: 0.7582 - accuracy: 0.8267
Epoch 1

In [13]:
# evaluate model using test data
model_loss, model_accuracy = nn.evaluate(X_test, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

12/12 - 0s - loss: 0.3851 - accuracy: 0.8723 - 174ms/epoch - 14ms/step
Loss: 0.3850504755973816, Accuracy: 0.8722826242446899


Looking at our deep learning model's performance metrics, the model was able to correctly identify employees who are at risk of attrition approximately 87% of the time. Considering that our input data included more than 30 different variables with more than 1,400 data points, the deep learning model was able to produce a fairly reliable classifier.

In [14]:
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 448       
                                                                 
 dense_1 (Dense)             (None, 5)                 45        
                                                                 
 dense_2 (Dense)             (None, 1)                 6         
                                                                 
Total params: 499
Trainable params: 499
Non-trainable params: 0
_________________________________________________________________


Neural networks, especially complex neural networks, are resource-hungry algorithms. When it comes to training neural networks on medium to large datasets, the amount of computation time to adequately train a model can take hours (or even days!) With simple modelling problems, like the ones covered in this module, training a model in the same notebook as an analysis is no problem. However, with more formal applications of neural network and deep learning models, data scientists cannot afford the time or resources to build and train a model each time they analyze data. In these cases, a trained model must be stored and accessed outside of the training environment.

With TensorFlow, we have the ability to save and load neural network models at any stage, including partially trained models. When building a TensorFlow model, if we use Keras' ModelCheckpoint method, we can save the model weights after it tests a set number of data points. Then, at any point, we can reload the checkpoint weights and resume model training. Saving checkpoints while training has a number of benefits:

-We can short-circuit our training loop at any time (stop the function by pressing CTRL+C, or by pressing the stop button at the top of the notebook). This can be helpful if the model is showing signs of overfitting.
-The model is protected from computer problems (power failure, computer crash, etc.). Worst-case scenario: We would lose five epochs' worth of optimization.
- We can restore previous model weight coefficients to try and revert overfitting.

Let's practice generating checkpoint files and loading model weights from different epochs. To make things simple, we'll implement checkpoints to our previous deep learning example notebook. To start, we'll open our "DeepLearning_Tabular" (or whatever similar name you may have used) notebook and rerun all of the code in the notebook for the following steps:

    Import dependencies.
    Import the input dataset.
    Generate categorical variable list.
    Create a OneHotEncoder instance.
    Fit and transform the OneHotEncoder.
    Add the encoded variable names to the DataFrame.
    Merge one-hot encoded features and drop the originals.
    Split the preprocessed data into features and target arrays.
    Split the preprocessed data into training and testing dataset.
    Create a StandardScaler instance.
    Fit the StandardScaler.
    Scale the data.
    Define the model.
    Add first and second hidden layers.
    Add the output layer.
    Check the structure of the model.

Now that we have our training data and our model defined, we're ready to compile and train our model using checkpoints. To use checkpoints, we need to define the checkpoint file name and directory path. For our purposes, we'll label our checkpoints by epoch number and contain them within their own folder. This ensures that our checkpoint files are neat, organized, and easily identifiable.


In [15]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

Once we have defined the file structure and filepath, we need to create a **callback** object for our deep learning model. A callback object is used in the Keras module to define a set of functions that will be applied at specific stages of the training process. There are a number of different callback functions available that can create log files, force training to stop, send training status messages, or in our case save model checkpoints. To create an effective checkpoint callback using the ModelCheckpoint method, we need to provide the following parameters:

- `filepath=checkpoint_path`—the checkpoint directory and file structure we defined previously
- `verbose=1`—we'll be notified when a checkpoint is being saved to the directory
- `save_weights_only=True`—saving the full model each time can fill up a hard drive very quickly; this ensures that the checkpoint files take up minimal space
- `save_freq='epoch'`—checkpoints will be saved every epoch


In [16]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch')

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
30/35 [========================>.....] - ETA: 0s - loss: 0.4684 - accuracy: 0.8323 
Epoch 1: saving model to checkpoints/weights.01.hdf5
35/35 [==============================] - 1s 2ms/step - loss: 0.4758 - accuracy: 0.8276
Epoch 2/100
33/35 [===========================>..] - ETA: 0s - loss: 0.4610 - accuracy: 0.8267
Epoch 2: saving model to checkpoints/weights.02.hdf5
35/35 [==============================] - 0s 3ms/step - loss: 0.4601 - accuracy: 0.8276
Epoch 3/100
29/35 [=======================>......] - ETA: 0s - loss: 0.4517 - accuracy: 0.8265
Epoch 3: saving model to checkpoints/weights.03.hdf5
35/35 [==============================] - 0s 2ms/step - loss: 0.4497 - accuracy: 0.8276
Epoch 4/100
31/35 [=========================>....] - ETA: 0s - loss: 0.4451 - accuracy: 0.8246
Epoch 4: saving model to checkpoints/weights.04.hdf5
35/35 [==============================] - 0s 2ms/step - loss: 0.4404 - accuracy: 0.8276
Epoch 5/100
27/35 [======================>.......] - ETA: 0

After running the previous code, we have created our trained model within the Python session, as well as a folder of checkpoints we can use to restore previous model weights. Now if we ever need to restore weights, we can use the Keras Sequential model's `load_weights` method to restore the model weights. To test this functionality, let's define another deep learning model, but restore the weights using the checkpoints rather than training the model.

In [17]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 8
hidden_nodes_layer2 = 5

nn_new = tf.keras.models.Sequential()

# First hidden layer
nn_new.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn_new.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn_new.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
nn_new.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Restore the model weights
nn_new.load_weights("checkpoints/weights.100.hdf5")

# Evaluate the model using the test data
model_loss, model_accuracy = nn_new.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

12/12 - 0s - loss: 0.5530 - accuracy: 0.8723 - 218ms/epoch - 18ms/step
Loss: 0.5529944896697998, Accuracy: 0.8722826242446899


Checkpoints are a great way to save model weights during training, but they fall short when it comes to sharing a trained model. In data science, trained models are published in scientific papers, deployed in software, open-sourced on GitHub, not to mention passed along to colleagues. In these cases, it is not practical to pass along only model weights, which can cause frustration and confusion. Instead, we can use the Keras Sequential model's save method to export the entire model (weights, structure, and configuration settings) to an Hierarchical Data Format ([HDF5](https://en.wikipedia.org/wiki/Hierarchical_Data_Format)) file. Once saved, anyone can import the exact same trained model to their environment by using the Keras `load_model` method and use it for analysis.

Even though we can save full neural network and deep learning models using Keras checkpoints, each full model file is almost ten times the size of a weight-only file. For those with limited hard drive space, saving full models using checkpoints is not feasible. We have a fully trained classification model that can predict employee attrition based on features in the dataset, so we can export it



In [18]:
# Export our model to HDF5 file
nn_new.save("trained_attrition.h5")

After running the code, we should see a file named "trained_attrition.h5," which contains the complete model and configuration. Now that we have the model saved, we can create the model at any point... like now.


In [21]:
# Import the model to a new object
nn_imported = tf.keras.models.load_model('trained_attrition.h5')

# Evaluate the completed model using the test data
model_loss, model_accuracy = nn_imported.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

12/12 - 0s - loss: 0.5530 - accuracy: 0.8723 - 156ms/epoch - 13ms/step
Loss: 0.5529944896697998, Accuracy: 0.8722826242446899


Looking at the final results, our imported model was able to reproduce the exact same performance metrics as the original model. Using this same procedure, we can import any type of Keras model for evaluation on a dataset with the same features.
